# Algo de novo em Python 3

Algumas surpresas para quem vem do Python 2:

In [12]:
L = [0, 1, 2]
zip(L, 'ABC')

In [3]:
map(lambda x: x*10, L)

In [18]:
filter(None, L)

Para construir listas, use o construtor `list`!

In [14]:
list(zip(L, 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [6]:
list(map(lambda x: x*10, L))

[10, 20, 30]

In [19]:
list(filter(None, L))

[1, 2]

Porém essess últimos exemplos eu faria assim:

In [7]:
[x*10 for x in L]

[10, 20, 30]

In [20]:
[x for x in L if x]

[1, 2]

E a versão *lazy* (preguiçosa), eu faria assim:

In [8]:
(x*10 for x in L)

<generator object <genexpr> at 0x105487a98>

A novidade é uso o ainda mais extensivo de geradores em Python 3.

# Como funciona um objeto gerador

Objetos geradores são iteradores: oferecem um método `__next__` que, quando acionado pela função embutida `next`, produzem o próximo item de uma série. A série pode ser limitada ou ilimitada.

In [21]:
z = zip(L, 'ABC')
z

In [22]:
next(z)

(0, 'A')

In [23]:
next(z)

(1, 'B')

In [24]:
next(z)

(2, 'C')

## Quando esgotado, um gerador levanta `StopIteration`

In [26]:
next(z)

StopIteration: 